# Preprocesamiento del Dataset FER2013

Este notebook demuestra el proceso de preprocesamiento de las imágenes del dataset FER2013 usando las funciones definidas en `src/data_prep.py`.

## Contenido:
1. Setup e importación
2. Exploración de datos originales
3. Aplicación del preprocesamiento
4. Verificación de resultados

In [ ]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import cv2
from src.data_prep import load_and_preprocess_images, save_processed_data, setup_processed_dirs

%matplotlib inline
plt.style.use('seaborn')

## 1. Funciones auxiliares para visualización

In [ ]:
def plot_sample_images(images, labels, num_samples=5):
    """Visualiza una muestra de imágenes con sus etiquetas"""
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
    for i in range(num_samples):
        axes[i].imshow(images[i], cmap='gray')
        axes[i].set_title(emotion_labels[labels[i]])
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

def plot_class_distribution(labels):
    """Visualiza la distribución de clases"""
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    unique, counts = np.unique(labels, return_counts=True)
    
    plt.figure(figsize=(10, 5))
    plt.bar(emotion_labels, counts)
    plt.title('Distribución de clases')
    plt.xticks(rotation=45)
    plt.ylabel('Número de imágenes')
    plt.show()

## 2. Exploración de datos originales

In [ ]:
# Cargar imágenes originales
raw_train_dir = Path('../data/raw/train')
print(f"Cargando imágenes de {raw_train_dir}...")

train_images, train_labels, train_filenames = load_and_preprocess_images(raw_train_dir)

# Visualizar algunas imágenes originales
print("\nMuestra de imágenes originales:")
plot_sample_images(train_images, train_labels)

# Mostrar distribución de clases
print("\nDistribución de clases en el conjunto de entrenamiento:")
plot_class_distribution(train_labels)

## 3. Aplicar preprocesamiento

In [ ]:
# Calcular estadísticas del conjunto de entrenamiento
train_mean = train_images.mean()
train_std = train_images.std()

print(f"Estadísticas del conjunto de entrenamiento:")
print(f"Media: {train_mean:.4f}")
print(f"Desviación estándar: {train_std:.4f}")

# Aplicar estandarización
train_images_processed = (train_images - train_mean) / train_std

# Visualizar algunas imágenes procesadas
print("\nMuestra de imágenes procesadas:")
plot_sample_images(train_images_processed, train_labels)

## 4. Guardar datos procesados

In [ ]:
# Configurar directorios de salida
processed_train_dir, processed_test_dir = setup_processed_dirs('../data')

# Guardar imágenes procesadas
print("Guardando imágenes procesadas...")
save_processed_data(train_images_processed, train_labels, train_filenames, processed_train_dir)

# Guardar estadísticas de normalización
stats = {
    'mean': train_mean,
    'std': train_std
}
np.save('../data/processed/normalization_stats.npy', stats)

print("\n¡Preprocesamiento completado!")
print(f"Imágenes guardadas en: {processed_train_dir}")
print(f"Estadísticas guardadas en: '../data/processed/normalization_stats.npy'")

## 5. Verificación final

In [ ]:
# Cargar y verificar algunas imágenes procesadas
processed_files = list(processed_train_dir.glob('*.jpg'))[:5]
processed_images = [cv2.imread(str(f), cv2.IMREAD_GRAYSCALE) for f in processed_files]
processed_images = np.array(processed_images) / 255.0

plt.figure(figsize=(15, 3))
for i, img in enumerate(processed_images):
    plt.subplot(1, 5, i+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
plt.suptitle('Verificación de imágenes procesadas guardadas')
plt.show()